In [1]:
import pandas as pd
import h5py
from pathlib import Path
import logging
import logging.config
import yaml
import h5py
import numpy as np
import matplotlib.pyplot as plt

import storage
import extraction

%matplotlib inline

In [2]:
def setupLogging():
    with open('../config/logConfig.yml', 'rt') as file:
        config = yaml.safe_load(file.read())
        logging.config.dictConfig(config)

In [3]:
setupLogging()

In [4]:
# Read TSV files

train_df = pd.read_table('../data/Train_GCC-training.tsv', header = None, names = ['caption', 'url'] )
#validate_df = pd.read_table('../data/Validation_GCC 1.1.0-Validation.tsv', header = None, names = ['caption', 'url']) 

train_df.head(20)

,caption,url
0,a very typical bus station,http://lh6.ggpht.com/-IvRtNLNcG8o/TpFyrudaT6I/...
1,sierra looked stunning in this top and this sk...,http://78.media.tumblr.com/3b133294bdc7c7784b7...
2,young confused girl standing in front of a war...,https://media.gettyimages.com/photos/young-con...
3,interior design of modern living room with fir...,https://thumb1.shutterstock.com/display_pic_wi...
4,cybernetic scene isolated on white background .,https://thumb1.shutterstock.com/display_pic_wi...
5,gangsta rap artist attends sports team vs play...,https://media.gettyimages.com/photos/jayz-atte...
6,the jetty : different types of plants to estab...,https://prismpub.com/wp-content/uploads/2016/1...
7,traditional ornamental floral paisley bandanna .,https://thumb1.shutterstock.com/display_pic_wi...
8,# of the sports team skates against sports tea...,https://media.gettyimages.com/photos/bryan-mcc...
9,by geographical feature category or in the cit...,http://www.robinhoodshow.com/clients/17668/864...


In [5]:
# Get last stored index
start_index = storage.get_last_stored_index("train")
print(start_index)

1004775


In [ ]:
# Extracts image and caption from tsv and store padded(image),caption and HTTPStatus code
# into a train or validation hdf5 file

# Set image size
size = 300 , 300 

# Extraction and Storage
#extraction.request_data_and_store(validate_df, size, "validate", start_index) 
extraction.request_data_and_store(train_df, size, "train", start_index - 100)
 

/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 12 bytes but only got 10. Skipping tag 42037
  " Skipping tag %s" % (size, len(data), tag)
/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:590: UserWarning: Metadata Warning, tag 45057 had too many entries: 2, expected 1
  % (tag, len(values))
/opt/conda/lib/python3.7/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [ ]:
# Reads the first couple captions and images stored in the validate/train hdf5 files
def retrieve(set_name): 
    
    for idx in range(10):
        try:
            status, image, caption = storage.read_image(set_name, idx)  
            if (int(status) == 200):

                print(image.shape)
                print(caption)
                plt.imshow(image)
                plt.show()
            
        except KeyError:
            # Ignores files not found - probably an HHTP error when requesting the URL
            # Later - Keep count of the minning files to know the exact size of the dataset
            print("Missing index - image not found - probably an HHTP error when requesting the URL " + str(idx))
            continue

    return

In [ ]:
retrieve("train")

In [ ]:
df = storage.get_statuses("train")
print(df["statuscode"].value_counts())

In [ ]:
df = storage.get_statuses("validation")

In [ ]:
print(df["statuscode"].unique().tolist())
print(df["statuscode"].value_counts().tolist())

plt.hist(df["statuscode"], bins = 18)
plt.style.use('ggplot')
plt.show()

print(df["statuscode"].value_counts())

In [ ]:

labels = df["statuscode"].value_counts().keys().tolist()
sizes = df["statuscode"].value_counts().tolist()
explode = np.zeros(18)
explode[1:18] = 0.1

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, explode=explode, radius=4000, shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

In [ ]:
from itertools import islice
df = pd.DataFrame(columns = ['statuscode'])
df.astype({'statuscode': 'int32'})
df["statuscode"] = ['a','b','c','d','e','f','g','h','i','j','k','l']

In [ ]:
for index, row in islice(df["statuscode"].iteritems(), 5, None):
    print(str(index) +"--"+ row)